# 1. Pre-steps
Including importing packages, login to the NASA Earthdata system, and get the token needed. Before you start, make sure you have registered an account in NASA Earthdata website: https://lpdaacsvc.cr.usgs.gov/appeears/

In [ ]:
import requests as r
import getpass, pprint, time, os, cgi, json
import geopandas as gpd

# Enter Earth Data login Credentials
username = getpass.getpass('Earthdata Username:')
password = getpass.getpass('Earthdata Password:')

api = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/'  # Set the AρρEEARS API to a variable

token_response = r.post('{}login'.format(api), auth=(username, password)).json() # Insert API URL, call login service, provide credentials & return json
del username, password                                                           # Remove user and password information
token_response 

# 2. Query available products
This section includes search available products and their layers. The most important thing in this section is to format the variable that stores products and layers we need. We will store this information in a variable named `prodLayer`, if you already know which product and layer to use, just formant the `prodLayer` variable as the example below.
## 2a. Search and explore available products

In [27]:
product_response = r.get('{}product'.format(api)).json()                         # request all products in the product service
print('AρρEEARS currently supports {} products.'.format(len(product_response)))  # Print no. products available in AppEEARS


AρρEEARS currently supports 119 products.


An example of a product's format in json:

In [28]:
products = {p['ProductAndVersion']: p for p in product_response} # Create a dictionary indexed by product name & version
products['MCD15A3H.006']                                         # Print information for MCD15A3H.006 LAI/FPAR Product

{'Product': 'MCD15A3H',
 'Platform': 'Combined MODIS',
 'Description': 'Leaf Area Index (LAI) and Fraction of Photosynthetically Active Radiation (FPAR)',
 'RasterType': 'Tile',
 'Resolution': '500m',
 'TemporalGranularity': '4 day',
 'Version': '006',
 'Available': True,
 'DocLink': 'https://doi.org/10.5067/MODIS/MCD15A3H.006',
 'Source': 'LP DAAC',
 'TemporalExtentStart': '2002-07-04',
 'TemporalExtentEnd': 'Present',
 'Deleted': False,
 'DOI': '10.5067/MODIS/MCD15A3H.006',
 'ProductAndVersion': 'MCD15A3H.006'}

We can search for specific products using keywords or strings. For example, here we search for products containing "Leaf Area Index" in the `Description`.

In [30]:
prodNames = {p['ProductAndVersion'] for p in product_response}
for p in prodNames:
    if "Leaf Area Index" in products[p]["Description"]:
        pprint.pprint(products[p])

{'Available': True,
 'DOI': '10.5067/VIIRS/VNP15A2H.001',
 'Deleted': False,
 'Description': 'Leaf Area Index (LAI) and Fraction of Photosynthetically '
                'Active Radiation (FPAR)',
 'DocLink': 'https://doi.org/10.5067/viirs/vnp15a2h.001',
 'Platform': 'S-NPP NASA VIIRS',
 'Product': 'VNP15A2H',
 'ProductAndVersion': 'VNP15A2H.001',
 'RasterType': 'Tile',
 'Resolution': '500m',
 'Source': 'LP DAAC',
 'TemporalExtentEnd': 'Present',
 'TemporalExtentStart': '2012-01-19',
 'TemporalGranularity': '8 day',
 'Version': '001'}
{'Available': True,
 'DOI': '10.5067/MODIS/MCD15A3H.006',
 'Deleted': False,
 'Description': 'Leaf Area Index (LAI) and Fraction of Photosynthetically '
                'Active Radiation (FPAR)',
 'DocLink': 'https://doi.org/10.5067/MODIS/MCD15A3H.006',
 'Platform': 'Combined MODIS',
 'Product': 'MCD15A3H',
 'ProductAndVersion': 'MCD15A3H.006',
 'RasterType': 'Tile',
 'Resolution': '500m',
 'Source': 'LP DAAC',
 'TemporalExtentEnd': 'Present',
 'TemporalEx

Now, using the above information, we can format the variable containing products we need. For example, here we select "MCD15A3H.006", "MOD11A2.006", and "SRTMGL1_NC.003":

In [31]:
prods = ["MCD15A3H.006", "MOD11A2.006", "SRTMGL1_NC.003"]

## 2b. Search and explore available layers
Next step is to get the layers we want from those products. We start from searching their all available layers.

In [32]:
lst_response = r.get("{}product/{}".format(api, prods[1])).json()
list(lst_response.keys())

['Clear_sky_days',
 'Clear_sky_nights',
 'Day_view_angl',
 'Day_view_time',
 'Emis_31',
 'Emis_32',
 'LST_Day_1km',
 'LST_Night_1km',
 'Night_view_angl',
 'Night_view_time',
 'QC_Day',
 'QC_Night']

If you want to check the information in a specific layer, you can do:

In [33]:
lst_response["LST_Day_1km"]

{'AddOffset': 0.0,
 'Available': True,
 'DataType': 'float32',
 'Description': 'Day Land Surface Temperature',
 'Dimensions': ['time', 'YDim', 'XDim'],
 'FillValue': 0,
 'IsQA': False,
 'Layer': 'LST_Day_1km',
 'OrigDataType': 'uint16',
 'OrigValidMax': 65535,
 'OrigValidMin': 7500,
 'QualityLayers': "['QC_Day']",
 'QualityProductAndVersion': 'MOD11A2.006',
 'ScaleFactor': 0.02,
 'Units': 'Kelvin',
 'ValidMax': 1310.699951,
 'ValidMin': 150.0,
 'XSize': 1200,
 'YSize': 1200}

Now, say we want 2 layers from this product: "LST_Day_1km" and "LST_Night_1km", we can do:

In [34]:
layers = [(prods[1], "LST_Day_1km"), (prods[1], "LST_Night_1km")]

Note: it is not required to store the product and layer information in a tupled list, but doing so wll make formatting json request later convinient.

We can use the same method to explore other products and their layers, now let's suppose we already done that and we got a tupled list that contains all products and layers we want:

In [36]:
layers.append((prods[0], "Lai_500m"))
layers.append((prods[2], "SRTMGL1_DEM"))
layers

[('MOD11A2.006', 'LST_Day_1km'),
 ('MOD11A2.006', 'LST_Night_1km'),
 ('MCD15A3H.006', 'Lai_500m'),
 ('SRTMGL1_NC.003', 'SRTMGL1_DEM')]

THen we can format this tupled list to a json variable, we store them in the variable `prodLayer` we mentioned earlier in this document.

In [37]:
prodLayer = []
for l in layers:
    prodLayer.append({
        "layer": l[1],
        "product": l[0]
    })
prodLayer

[{'layer': 'LST_Day_1km', 'product': 'MOD11A2.006'},
 {'layer': 'LST_Night_1km', 'product': 'MOD11A2.006'},
 {'layer': 'Lai_500m', 'product': 'MCD15A3H.006'},
 {'layer': 'SRTMGL1_DEM', 'product': 'SRTMGL1_NC.003'}]

# 3. Submit an "Area Request"
Now we have the products and layers we need, but before submitting the job to AppEEARS application, we need a region indicating our area of interest and a time period indicating the what are the time series we want.

## 3a. import a shapefile
Let's import a shapefile first, and subset the shapefile to the area of interest only, and convert it to json format.

In [44]:
nps = gpd.read_file("/Users/xgao26/Downloads/nps_boundary/nps_boundary.shp")
print(nps)

nps_gc = nps[nps["UNIT_NAME"] == "Grand Canyon National Park"].to_json()
nps_gc = json.loads(nps_gc)

UNIT_CODE                                          GIS_Notes  \
0        AMME  PRELIMINARY - Data has not completed the entir...   
1        CEBE  Lands - http://landsnet.nps.gov/tractsnet/docu...   
2        LIBI  Lands - http://landsnet.nps.gov/tractsnet/docu...   
3        CAVO  Lands - http://landsnet.nps.gov/tractsnet/docu...   
4        FOBO  Lands - http://landsnet.nps.gov/tractsnet/docu...   
..        ...                                                ...   
417      SLBE  Lands - http://landsnet.nps.gov/tractsnet/docu...   
418      RICH  Lands - http://landsnet.nps.gov/tractsnet/docu...   
419      APPA  Lands - http://landsnet.nps.gov/tractsnet/docu...   
420      OBRI  PRELIMINARY - Data has not completed the entir...   
421      SAMO  Lands - http://landsnet.nps.gov/tractsnet/docu...   

                                             UNIT_NAME   DATE_EDIT STATE  \
0                               American Memorial Park  2015-04-22    MP   
1    Cedar Creek and Belle Grove Na

## 3b. Search and explore available projections

In [45]:
projections = r.get('{}spatial/proj'.format(api)).json()  # Call to spatial API, return projs as json
projections

[{'Name': 'native',
  'Description': 'Native Projection',
  'Platforms': '',
  'Proj4': '',
  'Datum': '',
  'EPSG': '',
  'Units': '',
  'GridMapping': '',
  'Available': True},
 {'Name': 'geographic',
  'Description': 'Geographic',
  'Platforms': "['SRTM', 'ECOSTRESS', 'SSEBop ET', 'GPW', 'ASTER GDEM', 'NASADEM']",
  'Proj4': '+proj=longlat +datum=WGS84 +no_defs',
  'Datum': 'wgs84',
  'EPSG': 4326.0,
  'Units': 'degrees',
  'GridMapping': 'latitude_longitude',
  'Available': True},
 {'Name': 'sinu_modis',
  'Description': 'MODIS Sinusoidal',
  'Platforms': "['Combined MODIS', 'Terra MODIS', 'Aqua MODIS', 'S-NPP NASA VIIRS', 'Global WELD']",
  'Proj4': '+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs',
  'Datum': '',
  'EPSG': '',
  'Units': 'meters',
  'GridMapping': 'sinusoidal',
  'Available': True},
 {'Name': 'albers_weld_alaska',
  'Description': 'WELD Albers Equal Area Alaska',
  'Platforms': "['WELD']",
  'Proj4': '+proj=aea +lat_1=55 +lat_2=6

Let's store all projection names in a variable:

In [47]:
projs = {}
for p in projections:
    projs[p["Name"]] = p
list(projs.keys())

['native',
 'geographic',
 'sinu_modis',
 'albers_weld_alaska',
 'albers_weld_conus',
 'albers_ard_alaska',
 'albers_ard_conus',
 'albers_ard_hawaii',
 'easegrid_2_global',
 'easegrid_2_north',
 'laea_sphere_19']

Say we want to use the projection "geographic" here:

In [48]:
projs["geographic"]

{'Name': 'geographic',
 'Description': 'Geographic',
 'Platforms': "['SRTM', 'ECOSTRESS', 'SSEBop ET', 'GPW', 'ASTER GDEM', 'NASADEM']",
 'Proj4': '+proj=longlat +datum=WGS84 +no_defs',
 'Datum': 'wgs84',
 'EPSG': 4326.0,
 'Units': 'degrees',
 'GridMapping': 'latitude_longitude',
 'Available': True}

## 3c. Compile a json and submit the task request
OK, now we are ready to go. To submit a task request, we need to format the variables in json format and submit it to the AppEEARS system

In [59]:
# task_name = input("Enter a Task Name:")
task_name = "test"
task_type = ["point", "area"]
proj = projs["geographic"]["Name"]
outFormat = ["geotiff", "netcdf4"]

startDate = "07-01-2017"
endDate = "07-31-2017"

recurring = False

# format them into a json variable
task = {
    "task_type": task_type[1], # area sample task
    "task_name": task_name,
    "params": {
        "dates": [
        {
            "startDate": startDate,
            "endDate": endDate
        }],
        "layers": prodLayer,
        "output": {
            "format": {
                "type": outFormat[0] # geotiff format
            },
            "projection": proj
        },
        "geo": nps_gc
    }
}

To submit the task, we only need to post this json variable to AppEEARS api address:

In [60]:
token = token_response['token']                      # Save login token to a variable
head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request

task_response = r.post("{}task".format(api), json = task, headers = head).json()
task_response

{'task_id': 'cd69f101-982c-4c35-8c45-512e67f9e8b9', 'status': 'pending'}

Now the task has been submitted, once it is done, we can download the result. Actually, once the task has been sumitted successfully, you will receive a notification email. Also, you can check the task status by login to the website of AppEEARS or using a retrieve script as below:

In [61]:
task_id = task_response['task_id']                                               # Set task id from request submission
status_response = r.get('{}status/{}'.format(api, task_id), headers=head).json() # Call status service with specific task ID & user credentials
status_response      

{'task_id': 'cd69f101-982c-4c35-8c45-512e67f9e8b9',
 'updated': '2020-06-29T22:30:33.278375',
 'user_id': 'jacorygao@outlook.com',
 'progress': {'details': [{'desc': 'Initializing',
    'step': 1,
    'pct_complete': 100},
   {'desc': 'Downloading', 'step': 2, 'pct_complete': 100},
   {'desc': 'Subsetting', 'step': 3, 'pct_complete': 100},
   {'desc': 'Generating Files', 'step': 4, 'pct_complete': 100},
   {'desc': 'Extracting Stats', 'step': 5, 'pct_complete': 100},
   {'desc': 'Finalizing', 'step': 6, 'pct_complete': 0}],
  'summary': 95},
 'status_id': '707eae8b-8c80-47b2-bec9-d98c7613c91a',
 'status_type': 'task'}

# 4. Download a request
At this step, the task we submitted has completed, we need to download the resutls. But first, we must know which files to download:

In [62]:
bundle = r.get('{}bundle/{}'.format(api,task_id)).json()  # Call API and return bundle contents for the task_id as json
bundle

{'files': [{'sha256': '64287eaefa844410766f6400be442a220458cb06baa66afdf7c124861ada94de',
   'file_id': 'ebe7d05a-febe-4131-ad84-ade8a68a428e',
   'file_name': 'MCD15A3H.006_2017179_to_2017212/MCD15A3H.006_Lai_500m_doy2017181_aid0001.tif',
   'file_size': 13511,
   'file_type': 'tif'},
  {'sha256': '61d98140d71b2bce3cccb6af6ebb150905651ac79f3918bcd4c6269aadc9dc81',
   'file_id': 'e744ab4a-5251-4ca0-9bcf-999bcf93c8fa',
   'file_name': 'MCD15A3H.006_2017179_to_2017212/MCD15A3H.006_Lai_500m_doy2017185_aid0001.tif',
   'file_size': 13430,
   'file_type': 'tif'},
  {'sha256': 'b3e2aefffc0b75c1211330cae71d6add77a0a49e4d66753450103e47a27336a4',
   'file_id': '663f2148-3e9c-49e1-87ce-84e1c5bf988e',
   'file_name': 'MCD15A3H.006_2017179_to_2017212/MCD15A3H.006_Lai_500m_doy2017189_aid0001.tif',
   'file_size': 13706,
   'file_type': 'tif'},
  {'sha256': '10d92a1091f9bac168cd48a88762cd9a367a131d3f6a29b6200e944f27cd70ff',
   'file_id': '31a4e01a-cc2f-4328-9a35-a22217fee0f3',
   'file_name': 'MCD15

Store those files to a variable:

In [63]:
files = {}                                                       # Create empty dictionary
for f in bundle['files']: files[f['file_id']] = f['file_name']   # Fill dictionary with file_id as keys and file_name as values
files

{'ebe7d05a-febe-4131-ad84-ade8a68a428e': 'MCD15A3H.006_2017179_to_2017212/MCD15A3H.006_Lai_500m_doy2017181_aid0001.tif',
 'e744ab4a-5251-4ca0-9bcf-999bcf93c8fa': 'MCD15A3H.006_2017179_to_2017212/MCD15A3H.006_Lai_500m_doy2017185_aid0001.tif',
 '663f2148-3e9c-49e1-87ce-84e1c5bf988e': 'MCD15A3H.006_2017179_to_2017212/MCD15A3H.006_Lai_500m_doy2017189_aid0001.tif',
 '31a4e01a-cc2f-4328-9a35-a22217fee0f3': 'MCD15A3H.006_2017179_to_2017212/MCD15A3H.006_Lai_500m_doy2017193_aid0001.tif',
 '457a7453-cc78-47a2-9dce-dd407e044fc6': 'MCD15A3H.006_2017179_to_2017212/MCD15A3H.006_Lai_500m_doy2017197_aid0001.tif',
 '0261af1e-b390-4168-95b5-8359637d3821': 'MCD15A3H.006_2017179_to_2017212/MCD15A3H.006_Lai_500m_doy2017201_aid0001.tif',
 'a259142b-2b24-4781-a9bd-74833fc817a5': 'MCD15A3H.006_2017179_to_2017212/MCD15A3H.006_Lai_500m_doy2017205_aid0001.tif',
 '526270b5-733f-41cd-9672-6f1596d4bde3': 'MCD15A3H.006_2017179_to_2017212/MCD15A3H.006_Lai_500m_doy2017209_aid0001.tif',
 '53220094-c4e6-4715-bdbf-c5f7fe

Then, just download to wherever you want:

In [64]:
destDir = os.path.join("/Users/xgao26/Downloads/", task_name)
if not os.path.exists(destDir) : os.makedirs(destDir)

for f in files:
    dl = r.get('{}bundle/{}/{}'.format(api, task_id, f), stream=True)                                # Get a stream to the bundle file
    filename = os.path.basename(cgi.parse_header(dl.headers['Content-Disposition'])[1]['filename'])  # Parse the name from Content-Disposition header 
    filepath = os.path.join(destDir, filename)                                                       # Create output file path
    with open(filepath, 'wb') as f:                                                                  # Write file to dest dir
        for data in dl.iter_content(chunk_size=8192): f.write(data) 
print('Downloaded files can be found at: {}'.format(destDir))

Downloaded files can be found at: /Users/xgao26/Downloads/cd69f101-982c-4c35-8c45-512e67f9e8b9
